Grupo 3 - miniproyecto Deep Learning

# Datos y librerias

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [3]:
# Carga de datos de archivo .csv
#dataTraining = pd.read_csv('trainReg.csv')
#dataTesting = pd.read_csv('testReg.csv')
# Carga de datos desde github
dataTraining = pd.read_csv('https://raw.githubusercontent.com/oapatinop/DL_miniproyecto/main/trainReg.csv')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/oapatinop/DL_miniproyecto/main/testReg.csv')

In [4]:
dataTraining = dataTraining.set_index('ID').drop('V3', axis=1)
dataTraining.head()

,Y,V1,V2,V4,V5,V6,V7,V8,V9,V10,...,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90
ID,,,,,,,,,,,,,,,,,,,,,
1,2013,44.81144,0.83826,7.91314,10.94148,-0.04547,-15.16332,-10.47324,14.17212,10.57382,...,-8.44873,-230.30484,-40.94698,48.20025,-0.28694,155.76251,-56.23579,13.62599,123.92018,10.02845
2,1998,41.99180,7.99976,64.26707,16.54115,-9.28737,-40.73524,33.60440,9.18802,-6.79144,...,18.68972,-44.06062,52.37792,81.36093,-14.81111,151.66273,-120.61213,10.57519,-3.21078,-1.07438
3,1998,42.19196,2.23111,65.07719,24.99746,1.76100,6.66573,3.45778,-24.42779,-18.45069,...,-3.69878,-118.95712,54.15529,-23.32168,-9.65067,-83.83055,-141.17594,7.33084,-275.69714,2.35522
4,1998,39.28634,-1.85716,91.04190,9.08333,0.08502,-5.59216,65.62463,8.33105,11.70269,...,20.89044,-18.53135,176.09769,351.33669,3.44682,121.69156,-270.43989,12.51659,-140.88884,-0.23476
5,1998,40.36025,2.94918,53.83723,13.71369,-8.21964,-40.21636,21.22366,17.16742,0.14463,...,19.91979,34.59026,-69.83720,102.31946,8.08807,135.08089,-153.02327,4.09207,-68.33046,-6.19159


In [5]:
dataTesting = dataTesting.set_index('ID').drop('V3', axis=1)
dataTesting.head()

,V1,V2,V4,V5,V6,V7,V8,V9,V10,V11,...,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90
ID,,,,,,,,,,,,,,,,,,,,,
5972,35.74170,0.86615,1.44450,-13.87834,-9.57602,5.89475,-4.46036,-10.60501,-2.20779,-31.56674,...,10.78210,-183.63792,131.18868,37.68626,-4.48926,10.98139,-48.14549,-7.27992,-54.52826,4.81414
57116,35.31664,-0.47799,-12.27958,-29.32847,-2.38610,-7.27386,-1.87712,-3.74953,-3.38237,1.54788,...,53.92478,-238.77713,134.28149,81.19403,-4.81669,106.00178,3.34940,2.72328,156.94894,-8.44873
29076,43.25032,-6.31920,-2.56469,-7.59368,2.12862,23.29535,-0.34225,4.48761,4.93676,33.62086,...,40.95711,-269.59803,69.88747,29.65701,-0.12854,106.92919,-62.86550,30.12278,24.39843,-10.68316
83358,44.94220,-4.12303,-15.79205,7.15794,1.46007,-3.38280,-8.48270,12.69732,-1.68102,3.62442,...,0.81574,-243.09497,24.20334,-10.02719,19.25667,66.15602,-64.78646,9.09148,11.46055,-7.12136
84664,50.40813,-1.52061,-10.10142,-17.74876,-5.96768,-29.94136,-11.84237,5.90144,3.95996,6.07786,...,-3.58634,-61.67109,47.53814,139.04916,6.85904,-57.09356,-49.57683,-6.61453,22.55497,-4.39272


In [6]:
def guardar(ypred,nombre):
    ypred = pd.DataFrame(ypred,index=dataTesting.index, columns=['Y'])
    # Guardar predicciones en formato exigido en la competencia de kaggle
    archivo = nombre + '.csv'
    ypred.to_csv(archivo, index_label='ID')
    return ypred

In [7]:
def calculo_rmse(ypred,yTest,ypredTr,yTrain):
    RMSE = round(mse(ypred, yTest, squared = False))
    MSE = round(mse(ypred, yTest))
    #MAE = round(mae(ypred, yTest))

    RMSE_Tr = round(mse(ypredTr, yTrain, squared = False))
    MSE_Tr = round(mse(ypredTr, yTrain))
    #MAE_Tr = round(mae(ypredTr, yTrain))

    # Cálculo del desempeño del modelo utilizando el RMSE y el MAE
    print(f'El RMSE del modelo es de {RMSE}')
    print(f'El MSE del modelo es de {MSE}')
    #print(f'El MAE del modelo es de {MAE}')

    print(f'El RMSE del modelo en train es de {RMSE_Tr}')
    print(f'El MSE del modelo en train es de {MSE_Tr}')
    #print(f'El MAE del modelo en train es de {MAE_Tr}')

# Exploración de datos

In [8]:
dTr = dataTraining.copy()

In [9]:
dTr.describe(include="all").T

,count,mean,std,min,25%,50%,75%,max
Y,77779.0,2002.308129,10.811038,1926.00000,1998.000000,2006.00000,2010.000000,2014.00000
V1,77779.0,43.425185,6.128869,4.83688,40.060315,44.32385,47.900080,60.03401
V2,77779.0,-0.136720,4.370466,-69.68087,-2.612435,-0.06300,2.465950,23.81526
V4,77779.0,3.755313,17.609183,-165.22161,-7.016240,2.02210,12.776450,274.65858
V5,77779.0,-2.339768,14.483975,-121.47534,-10.685075,-2.05456,6.423900,160.81522
...,...,...,...,...,...,...,...,...
V86,77779.0,17.575097,115.243644,-3168.92457,-31.569390,15.23736,67.361475,2144.10391
V87,77779.0,-25.628857,173.310304,-4319.99232,-100.668480,-21.58164,51.333150,2833.60895
V88,77779.0,4.463484,13.526414,-236.03926,-2.569360,3.13690,10.002145,275.35366
V89,77779.0,18.664885,186.690183,-7458.37815,-60.121980,5.94149,84.389125,5289.11138


In [10]:
sum(dTr.isnull().sum() / dTr.shape[0] * 100)

0.0

# Preparación de datos

In [11]:
XTotal = dTr.loc[:,dTr.columns!='Y']
yTotal = dTr.loc[:,dTr.columns=='Y']

In [12]:
scaler = StandardScaler(with_mean=True, with_std=True)
scaler.fit(XTotal)
XTotalS=scaler.transform(XTotal)
dTestS=scaler.transform(dataTesting)

In [13]:
XTrain, XTest, yTrain, yTest = train_test_split(XTotalS, yTotal, test_size=0.30, random_state=3)

# Análisis preliminar de modelos

## Keras

In [17]:
knn = Sequential()
knn.add(Dense(256, activation='relu', input_shape=(XTrain.shape[1],)))
knn.add(Dropout(0.2))
knn.add(Dense(256, activation='relu'))
knn.add(Dropout(0.2))
knn.add(Dense(256, activation='relu'))
knn.add(Dense(1))
# Compilación del modelo
knn.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamiento del modelo
knn.fit(XTrain, yTrain.to_numpy(), epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
1362/1362 [==============================] - 10s 6ms/step - loss: 246049.5781 - val_loss: 41608.7891
Epoch 2/50
1362/1362 [==============================] - 13s 9ms/step - loss: 31487.5977 - val_loss: 10765.3447
Epoch 3/50
1362/1362 [==============================] - 14s 10ms/step - loss: 16681.1230 - val_loss: 3413.1453
Epoch 4/50
1362/1362 [==============================] - 14s 10ms/step - loss: 13093.9600 - val_loss: 4434.4434
Epoch 5/50
1362/1362 [==============================] - 11s 8ms/step - loss: 11673.3086 - val_loss: 3186.2720
Epoch 6/50
1362/1362 [==============================] - 7s 5ms/step - loss: 10121.5098 - val_loss: 2991.7615
Epoch 7/50
1362/1362 [==============================] - 6s 4ms/step - loss: 9423.8906 - val_loss: 5433.6030
Epoch 8/50
1362/1362 [==============================] - 7s 5ms/step - loss: 8408.0938 - val_loss: 1479.4414
Epoch 9/50
1362/1362 [==============================] - 6s 5ms/step - loss: 7703.0132 - val_loss: 926.2943
Epoch 10/50
1

In [18]:
keras64=knn.predict(XTest)
keras64Tr=knn.predict(XTrain)
calculo_rmse(keras64,yTest,keras64Tr,yTrain)

1702/1702 [==============================] - 3s 2ms/step
El RMSE del modelo es de 16
El MSE del modelo es de 262
El RMSE del modelo en train es de 16
El MSE del modelo en train es de 266


In [19]:
kerasNN1=knn.predict(dTestS)
guardar(kerasNN1,'KerasNN1')

584/584 [==============================] - 1s 2ms/step


,Y
ID,
5972,1992.640381
57116,1987.909058
29076,1987.371216
83358,1988.791138
84664,1993.279297
...,...
5763,1992.640381
25441,1995.368896
53318,1990.993286


In [67]:
# Definición del modelo
knn = Sequential()
knn.add(Dense(128, input_dim=90, activation='relu'))  # Capa oculta 1 con 64 neuronas
knn.add(Dense(128, activation='relu'))               # Capa oculta 2 con 64 neuronas
knn.add(Dense(128, activation='relu'))               # Capa oculta 3 con 64 neuronas
knn.add(Dropout(0.2))
knn.add(Dense(1))             # Capa de salida con 1 neurona (regresión)

# Compilación del modelo
knn.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamiento del modelo
knn.fit(XTrain, yTrain.to_numpy(), epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1362/1362 [==============================] - 7s 3ms/step - loss: 334019.2812 - val_loss: 52868.5078
Epoch 2/10
1362/1362 [==============================] - 6s 4ms/step - loss: 46771.9414 - val_loss: 15616.2676
Epoch 3/10
1362/1362 [==============================] - 7s 5ms/step - loss: 22306.2559 - val_loss: 5938.3901
Epoch 4/10
1362/1362 [==============================] - 7s 5ms/step - loss: 18464.4297 - val_loss: 2291.5510
Epoch 5/10
1362/1362 [==============================] - 4s 3ms/step - loss: 18231.5176 - val_loss: 3076.6348
Epoch 6/10
1362/1362 [==============================] - 4s 3ms/step - loss: 17970.1367 - val_loss: 2401.2449
Epoch 7/10
1362/1362 [==============================] - 4s 3ms/step - loss: 17875.7422 - val_loss: 1978.1670
Epoch 8/10
1362/1362 [==============================] - 5s 3ms/step - loss: 17667.1328 - val_loss: 2048.9641
Epoch 9/10
1362/1362 [==============================] - 4s 3ms/step - loss: 17800.4766 - val_loss: 1638.4186
Epoch 10/10
1362

In [68]:
keras64=knn.predict(XTest)
keras64Tr=knn.predict(XTrain)
calculo_rmse(keras64,yTest,keras64Tr,yTrain)

1702/1702 [==============================] - 3s 1ms/step
El RMSE del modelo es de 47
El MSE del modelo es de 2253
El RMSE del modelo en train es de 46
El MSE del modelo en train es de 2141


In [ ]:
# Definición red neuronal con la función Sequential()
gru = tf.keras.Sequential()

# Definición de la capa embedding
gru.add(Embedding(XTrain.shape[0], 128, input_length=XTrain.shape[1]))
# Definición de la capa recurrente LSTM
gru.add(GRU(64))
# Definición de dropout para evitar overfitting
gru.add(Dropout(0.2))
# Definición capa densa con función sigmoide para predicción binaria final
gru.add(Dense(yTrain.shape[1], activation='softmax'))

# Definición de función de perdida.
gru.compile(loss='mean_squared_error', optimizer='adam')

# Impresión de la arquitectura de la red neuronal
gru.summary()

## Scikit Learn

In [14]:
sklnn = MLPRegressor(hidden_layer_sizes=(128, 128, 128), activation='relu', solver='lbfgs',
                     max_iter=500, random_state=42)
sklnn.fit(XTrain, yTrain)

MLPRegressor(hidden_layer_sizes=(128, 128, 128), max_iter=500, random_state=42,
             solver='lbfgs')

In [15]:
sklNN=sklnn.predict(XTest)
sklNNTr=sklnn.predict(XTrain)

In [16]:
calculo_rmse(sklNN,yTest,sklNNTr,yTrain)

El RMSE del modelo es de 12
El MSE del modelo es de 138
El RMSE del modelo en train es de 9
El MSE del modelo en train es de 77


## Random Search Scikit learn

In [20]:
param_test = {
    'hidden_layer_sizes':[(64,64),(128,128,128),(256,256)],
    'activation':['identity', 'logistic', 'tanh', 'relu'],
    'solver':['lbfgs', 'sgd', 'adam']
}
RS_skl = RandomizedSearchCV(estimator = MLPRegressor(max_iter=200, random_state=3),
                        param_distributions = param_test, scoring='neg_mean_squared_error',n_iter=5,n_jobs=-1, cv=5, verbose=True)
RS_skl.fit(XTrain,yTrain)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


RandomizedSearchCV(cv=5, estimator=MLPRegressor(random_state=3), n_iter=5,
                   n_jobs=-1,
                   param_distributions={'activation': ['identity', 'logistic',
                                                       'tanh', 'relu'],
                                        'hidden_layer_sizes': [(64, 64),
                                                               (128, 128, 128),
                                                               (256, 256)],
                                        'solver': ['lbfgs', 'sgd', 'adam']},
                   scoring='neg_mean_squared_error', verbose=True)

In [21]:
RS_skl.best_params_ , RS_skl.best_score_

({'solver': 'lbfgs', 'hidden_layer_sizes': (64, 64), 'activation': 'logistic'},
 -87.07201952336706)

# Entrenamiento y calibración

In [ ]:
predNN1=sklnn.predict(dTestS)
guardar(predNN1,'Skl_NN4')

,Y
ID,
5972,2005.826770
57116,1992.707868
29076,1999.214673
83358,2001.212872
84664,2005.111924
...,...
5763,2011.702906
25441,2000.864693
53318,1997.623429


# Visualización de resultados